In [13]:
# magic! (don't worry about this)
%load_ext autoreload
%autoreload 2

In [2]:
# let us import some useful things
from lib import *
from classifiers import *
import math
from scipy.stats import ttest_ind
from scipy.special import stdtr
import numpy as np

In [3]:
# load the data same as before
microarray_file_name = '../data/leukemia_ALL_AML_matrix.txt'
labels_file_name = '../data/leukemia_ALL_AML_labels.txt'
data_store = DataSet(microarray_file_name, labels_file_name)

In [10]:
# first let's combine our train and test sets to use as much data as possible
full_data_set = data_store.get_train_set() + data_store.get_test_set()

# separate samples based on label
ALL_gene_profiles_list = [s.get_gene_profile() for s in full_data_set if s.get_label() == 1]
AML_gene_profiles_list = [s.get_gene_profile() for s in full_data_set if s.get_label() == 0]


# let's select our most useful genes based on the Welch's t-test!
init_number_of_genes = len(full_data_set[0].get_gene_profile())
selected_genes = []
for gene_id in range(init_number_of_genes):
    gene_values_ALL_samples = [profile[gene_id] for profile in ALL_gene_profiles_list]
    gene_values_AML_samples = [profile[gene_id] for profile in AML_gene_profiles_list]
    
    # TODO: use the formulas we've seen to compute the t-test value and the degree of freedom for the ALL and AML vector
    # hint: to compute the mean and variance of each list -- use the np functions mean() and var()!
    
    t_test_val = 0 # TODO: change this line!
    degree_of_freedom = 0 # TODO: change this line!

    # now we can use an existing funtion to find the p-value (don't worry about this!)
    p_value = 2*stdtr(degree_of_freedom, -np.abs(t_test_val))
    
    if(p_value < 0.05):
        selected_genes.append(gene_id)

print 'Number of informative genes: ' , len(selected_genes)

# now let's update our dataset to ignore the genes that are not informative
filtered_samples = []
for sample in full_data_set:
    new_sample = Sample(sample.get_label(), [sample.get_gene_profile()[i] for i in range(init_number_of_genes) if i in selected_genes])
    filtered_samples.append(new_sample)
    
train_size = int(len(filtered_samples) * 0.66) # 2/3 of the data
train_set = filtered_samples[:train_size]
test_set = filtered_samples[train_size:]

k = 3
knn = KNearestNeighbors()
knn.train(train_set) # provide training samples
classified_samples = knn.classify(test_set, k, euclidean_distance)

# let's evaluate how well the classifier worked
evaluate_results(classified_samples)
    

Number of informative genes:  0
Accuracy: 72.00%
